<a href="https://colab.research.google.com/github/amirda9/DeepLearningCourse/blob/master/Part1_Amir_Naeini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hi 
My name is Amirmohammad Naeini (Amir) and I did this with my type of architecture because it was easier and needed less code. 
Thanks

Import some libraries

In [ ]:
from torchvision import datasets
import numpy as np
from PIL import Image
# import pudb
import random
%matplotlib notebook
from matplotlib import pyplot as plt

We provide you with this utility to shuffle and batch your data for you

In [ ]:
class Simple_batcher:
    '''
    A simple loader that shuffles data, and retrieves batches
    This is provided to you since the data input pipeline is not the focus of this part
    '''
    def __init__(self,dataset,batch_size):
        self.dataset = dataset
        self.batch_size = batch_size
        self.init_shuffling()

    def init_shuffling(self):
        '''
        Generates a shuffled order of the data idxs, resets the index for that shuffled list
        '''
        self.shuffled_idxs = list(range(len(dataset)))
        random.shuffle(self.shuffled_idxs)
        self.current_position = 0

    def get_next_pos(self):
        '''
        Grabs the next index from the shuffled list, when we reach the end, shuffle again
        '''
        pos = self.current_position
        self.current_position += 1
        if self.current_position >= len(self.dataset):
            self.init_shuffling()
        return pos

    def get_batch(self):
        '''
        Gathers multiple images and labels (as logits) into a batch for training
        MNIST images are single channel 28x28, uint8 images
        This dataloader will flatten each image into a 28*28 vector, and convert it into a float32

        N - Batch size
        Returned image shape: [N,784], dtype=float32
        Returned label shape: [N,1], dtype=int32
        '''
        batch_size = self.batch_size
        out_ims = []
        out_logits = []
        for n in range(batch_size):
            example = self.dataset[self.get_next_pos()]
            im = np.asarray(example[0]).flat
            label = example[1]
            out_ims.append(im)
            out_logits.append(label)
        out_ims = np.stack(out_ims,axis=0).astype(np.float32)
        out_logits = np.array([out_logits]).transpose(1,0)
        return out_ims, out_logits

**Here goes our function to compute accuracy**




In [ ]:
def compute_accuracy(prediction,gt_logits):
    # print(gt_logits)
    accuracy = np.count_nonzero(prediction==gt_logits)
    # print(accuracy,len(prediction))
    return accuracy

Setup for the components we will use
We will use the MNIST training dataset provided by torch

In [ ]:
class Net:
    def __init__(self,input_shape,output_shape,learning_rate,batch_size):
        '''
        Construct all layers, and a loss function
        '''
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.lr = learning_rate
        self.init_weights()
        self.batch_size = batch_size

    def init_weights(self):
        # self.W1 = np.random.randn(self.input_shape,256) 
        # self.W2 = np.random.randn(self.W1.shape[1],128) 
        # self.W3 = np.random.randn(self.W2.shape[1],10) 

        'Making our network'
        self.W1 = np.random.uniform(-1/np.sqrt(self.input_shape),1/np.sqrt(self.input_shape),(self.input_shape,300))
        self.W2 = np.random.uniform(-1/np.sqrt(self.W1.shape[1]),1/np.sqrt(self.W1.shape[1]),(self.W1.shape[1],150))
        self.W3 = np.random.uniform(-1/np.sqrt(self.W2.shape[1]),1/np.sqrt(self.W2.shape[1]),(self.W2.shape[1],10))


        self.b1 = np.zeros(self.W1.shape[1],) 
        self.b2 = np.zeros(self.W2.shape[1],) 
        self.b3 = np.zeros(self.W3.shape[1],) 

    'making grad zero'
    def zero_grads(self):
        self.grad=0

    'softmax calculation with avoiding nan and inf'
    def softmax(self,z):
        return np.exp(z)/np.sum(np.exp(z),axis=1).reshape(z.shape[0],1)


    "Relu and Its derivative"
    def ReLU(self,x):
        out = np.maximum(0,x)
        return out

    def dReLU(self,x):
        return 1 * (x > 0) 

    'Our cross entropy loss function '
    def crossEntropy_loss(self,x,real):
        # softmax =  np.exp(x)/np.sum(np.exp(x),axis=1).reshape(x.shape[0],1)
        entropy = np.zeros((x.shape[0]))
        for i in range(entropy.shape[0]):
            # print(x[i][real[i]])
            entropy[i] = (-np.log(x[i][real[i]]))
        return entropy

    "If it is a linear, we use this for grad"
    def dense_backward(self,input,grad_output,weights):
        grad_input = np.dot(grad_output,np.transpose(weights)) # that's where the magic happens
        dw = np.transpose(np.dot(np.transpose(grad_output),input))
        db = np.sum(grad_output,axis=0)
        return dw,db,grad_input

    def dense_forward(self,input,weights,bias):
        temp = input.dot(weights) + bias
        return temp

    "If it is a Relu, we use this for grad"
    def Relu_backward(self,input,grad_output):
        relu_grad = 1 * (input > 0) 
        # print(relu_grad)
        return relu_grad * grad_output

    "Network Forward"
    def forward(self,x):
        self.x = x 
        self.z1 = self.dense_forward(x,self.W1,self.b1)
        self.a1 = self.ReLU(self.z1)
        self.z2 = self.dense_forward(self.a1,self.W2,self.b2)
        self.a2 = self.ReLU(self.z2)
        self.z3 = self.dense_forward(self.a2,self.W3,self.b3)
        self.a3 = self.softmax(self.z3)
        return self.a3 

    # computing loss, pred and grad_ctx
    def forward_with_loss(self,x,labels):
        "pred_vec is softmax and network output"
        pred_vec = self.forward(x)
        # softmax = self.softmax(pred_vec) 
        loss_vec = self.crossEntropy_loss(pred_vec,labels)
        loss = np.mean(loss_vec)
        pred = (np.argmax(pred_vec,axis=1)).reshape(labels.shape)

        one_hot = np.zeros_like(pred_vec)
        for j in range(pred_vec.shape[0]):
            one_hot[j][labels[j]] = 1
        grad_ctx = (- one_hot + pred_vec) # the most important variable
        
        # pred = np.zeros_like(labels)
        # for j in range(pred_vec.shape[0]):
        #     pred[j]=np.argmax(pred_vec[j])
        # loss = np.mean(grad_ctx**2)
        # print(pred)
        return pred,loss,grad_ctx


    "Lets do some serious stuff"
    def compute_grads(self,grad_ctx):
        dw3,db3,grad_ctx = self.dense_backward(self.a2,(1/self.batch_size)*grad_ctx,self.W3)
        grad_ctx = self.Relu_backward(self.z2,grad_ctx)
        dw2,db2,grad_ctx = self.dense_backward(self.a1,grad_ctx,self.W2)
        grad_ctx = self.Relu_backward(self.z1,grad_ctx)
        dw1,db1,grad_ctx= self.dense_backward(self.x,grad_ctx,self.W1)


        grad = {
            'dw3':dw3,
            'dw2':dw2,
            'dw1':dw1,
            'db3':db3,
            'db2':db2,
            'db1':db1,
        }
        self.grad = grad


        "We are basically implementing this (Of course we need so many np.transpose for this to work)"
        # last layer weights grad = mat mul of (grad_ctx,last layer input)
        # second layer weights grad = mat mul of ((mat mul of ((grad_ctx), last layer weights) * d of Relu (second layer out before relu)), output of first layer)
        # first layer weights grad = may mul of ((mat mul of (mat mul of ((grad_ctx),last layer weights)*d of relu (second layer out before relu),weights of second layer)*d of relu( first layer out before relu)) , input)
        "Lets do that for biases"
        # bias grad of last layer  = sum(grad_ctx ,axis = 0)
        # bias grad of second layer = sum(mat mul of((grad_ctx),last layer weights) * d of relu(second layer out before relu),axis = 0)
        # bias grad of first layer = sum((mat mul of (mat mul of((grad_ctx),last layer weights)*d of relu(second layer out before relu),second layer weights)*d of relu(first layer out before relu)),axis = 0)
    
    'There we go'
    def sgd_step(self,learning_rate):
        self.W3 -=  learning_rate * self.grad['dw3']
        self.W2 -= learning_rate * self.grad['dw2']
        self.W1 -= learning_rate * self.grad['dw1']

        self.b3 -= learning_rate * self.grad['db3']
        self.b2 -= learning_rate * self.grad['db2']
        self.b1 -= learning_rate * self.grad['db1']
        

In [ ]:
batch_size = 100
learning_rate = 0.05
dataset = datasets.MNIST('data', train=True, download=True)
loader = Simple_batcher(dataset,batch_size)
net = Net(input_shape=784,output_shape=1,learning_rate=learning_rate,batch_size = batch_size)
losses = [] # store losses throughout training
accuracies = [] # store accuracies throughout training

Training loop, provided to you

In [ ]:
"This part is very similar to the skeleton"
for it in range(1000):

    net.zero_grads() # zero out the gradients
    ims, logits = loader.get_batch() # get the data
    ims = ims/127.5 - 1.0 # preprocessing, we just make the data [-1,1]

    pred, loss, grad_ctx = net.forward_with_loss(ims,logits) # do the forward pass
    accuracy = compute_accuracy(pred,logits) # compute accuracy
    losses.append(loss) # store loss
    accuracies.append(accuracy) # store accuracy
    if it > 50:
        learning_rate = net.lr - it*0.0000001
        pass
    net.compute_grads(grad_ctx) # backward pass
    net.sgd_step(learning_rate) # take a step of SGD

    if it % 10 == 0: # print the loss every 10 iterations
        print(loss,accuracy)

2.3020466736196425 13
2.208838041414819 35
2.0971292687113547 42
1.9883919676679895 49
1.589132471355639 71
1.315812567327156 65
0.973915553129317 75
0.9617890455358815 70
0.7849987097101975 85
0.6926271894132775 77
0.7007293610835761 82
0.4549428912856199 90
0.5529150214447123 86
0.8315536936245062 72
0.7157108467345893 73
0.40171958935645996 91
0.6557267358624754 84
0.5345874760384404 84
0.4540113441686247 89
0.48104436073271484 86
0.8134803595892586 71
0.45501605033407727 91
0.37464154431011887 87
0.4331540234985423 88
0.438564187962271 84
0.43191614005036866 86
0.3534510489727317 86
0.2596339642226567 95
0.199550084621304 97
0.4083955328293301 87
0.4444940685129881 82
0.4677977167928222 83
0.363700685242608 85
0.37136308582632116 89
0.3460716808808752 93
0.2797301511574652 93
0.39181277926184094 92
0.40665310218559775 87
0.34163632707490776 88
0.2335422870676091 94
0.3775861858653382 86
0.41253368769855314 89
0.3481928897367969 90
0.3928777374029866 87
0.2996532179080101 88
0.54299

Post training stuff, plot the stats from training
The non-notebook version will save the images

In [ ]:
print('Training complete, plotting')

plt.ioff() # stop notebook from drawing to the same figure
plt.plot(losses)
plt.title('Loss Plot')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.savefig('training-loss.png')
plt.show()

Training complete, plotting


<IPython.core.display.Javascript object>

In [ ]:
plt.plot(accuracies)
plt.title('Accuracy Plot')
plt.xlabel('Iteration')
plt.ylabel('Accuracy')
plt.savefig('training-acc.png')
plt.show()

<IPython.core.display.Javascript object>